In [ ]:
!pip install textblob

import pandas as pd
from textblob import TextBlob

In [ ]:
# Trigger output file (jisme trigger_predicted_label aa chuka hai)
df = pd.read_csv("sentiment_output.csv")

df.head()

,Unnamed: 0,id,title,rating,user/userLocation/shortName,placeInfo/id,placeInfo/name,placeInfo/rating,reply,cleaned_review,cleaned_reply,cleaned_review2,cleaned_response2,trigger_predicted,trigger_predicted_label,review_sentiment,sentiment_label
0,0,0,Kids club,5,NaN,307512,"ITC Grand <LOCATION>, a Luxury Collection Reso...",4.6,"Dear Valued Guest,\n\nNamaste,\n\nThank you fo...",It was very pleasant and wonderful stay specia...,"Dear Valued Guest, Namaste, Thank you for taki...",It was very pleasant and wonderful stay specia...,"Dear Valued Guest, <LOCATION> Thank you for ta...",0,short_reply,0.8874,positive
1,1,1,Excellent,5,NaN,620076,<LOCATION> Marriott Hotel & Convention Centre,4.4,"Dear Guest, \nThank you for your wonderful fee...",The hotel is fabulous and especially the Tattv...,"Dear Guest, Thank you for your wonderful feedb...",The hotel is fabulous and especially the Tattv...,"Dear Guest, Thank you for your wonderful feedb...",1,long_reply,0.9421,positive
2,2,2,Good visit,4,London,299754,JW Marriott Mumbai Juhu,4.5,"Dear Tripadvisor Member,\n\nThank you for taki...",Good experience at the JE Marriott Juhu. Servi...,"Dear Tripadvisor Member, Thank you for taking ...",Good experience at the JE Marriott Juhu. Servi...,"Dear Tripadvisor Member, Thank you for taking ...",0,short_reply,0.4767,positive
3,3,3,Excellent stay,5,NaN,8435492,Seclude Mussoorie,4.5,"Dear Guest,\nThank you for sharing your experi...",We came in group during our vacations in June ...,"Dear Guest, Thank you for sharing your experie...",We came in group during our vacations in June ...,"Dear Guest, Thank you for sharing your experie...",0,short_reply,0.9348,positive
4,4,4,"The hotel is neat very well maintained, the fo...",5,NaN,1141615,Gulf Hotel,3.9,"Dear Guest,\n\nGreetings from Gulf Hotel\n\nWe...","The best hotel in Mumbai as per my opinion, th...","Dear Guest, Greetings from HOTEL NAME We welco...",The best hotel in <LOCATION> as per my opinion...,"Dear Guest, Greetings from <HOTEL_NAME> We wel...",0,short_reply,0.9522,positive


In [ ]:
df.columns

Index(['Unnamed: 0', 'id', 'title', 'rating', 'user/userLocation/shortName',
       'placeInfo/id', 'placeInfo/name', 'placeInfo/rating', 'reply',
       'cleaned_review', 'cleaned_reply', 'cleaned_review2',
       'cleaned_response2', 'trigger_predicted', 'trigger_predicted_label',
       'review_sentiment', 'sentiment_label'],
      dtype='object')

In [ ]:
def get_sentiment_label(rating, sentiment):
    # Safety: handle NaN
    if pd.isna(rating):
        return "unknown"

    if rating >= 4:
        return "positive"
    elif rating <= 2:
        return "negative"
    else:  # rating == 3
        if sentiment >= 0:
            return "positive"
        else:
            return "negative"

df["sentiment_label"] = df.apply(
    lambda row: get_sentiment_label(row["rating"], row["review_sentiment"]),
    axis=1
)

df["sentiment_label"].value_counts()

,count
sentiment_label,
positive,7633
negative,2319


In [ ]:
print(df["sentiment_label"].unique())
print(df["trigger_predicted_label"].unique())

['positive' 'negative']
['short_reply' 'long_reply']


In [ ]:
df["sentiment_label"].unique().tolist()
df["trigger_predicted_label"].unique().tolist()

['short_reply', 'long_reply']

In [ ]:
def route_generator(sentiment, trigger_len):
    if sentiment >=0 and trigger_len == "short_reply":
        return "pos_short"
    elif sentiment >=0 and trigger_len == "long_reply":
        return "pos_long"
    elif sentiment <0 and trigger_len == "short_reply":
        return "neg_short"
    elif sentiment <0 and trigger_len == "long_reply":
        return "neg_long"
    else:
        return "unknown"

df["final_generator"] = df.apply(
    lambda row: route_generator(row["review_sentiment"], row["trigger_predicted_label"]),
    axis=1
)

df["final_generator"].value_counts()

,count
final_generator,
pos_short,4402
pos_long,4049
neg_long,975
neg_short,526


In [ ]:
print(df.shape)

(9952, 18)


In [ ]:
df.to_csv("/content/routing_output.csv", index=False)
print("Routing complete!")

Routing complete!
